## Quantile score

In [16]:
import numpy as np
import pandas as pd

from forecast_tools.baseline import Naive1

from typing import Union

In [75]:
def quantile_score(dist: Union[np.ndarray, list[float]],
                   actual: float,
                   probs: list[float] = [0.05, 0.25, 0.5, 0.75, 0.95],
                   na_rm: bool = True) -> float:
    """
    Calculate Quantile Score for probabilistic forecasts
    
    Parameters:
    ----------
    dist : array-like
        forecast samples
    actual : float
        observed value
    probs : list, optional (default = [0.05, 0.25, 0.5, 0.75, 0.95])
        list of probabilities to evaluate
    na_rm : bool, optional (default = True)
        remove missing values
    
    Returns:
    Quantile score (2 * mean loss across all quantiles)
    """
    # Convert to numpy array and handle NA values
    dist_array = np.asarray(dist)

    print(dist_array)
    
    if na_rm:
        dist_array = dist_array[~np.isnan(dist_array)]
    
    # Handle edge cases
    if len(dist_array) == 0 or np.isnan(actual):
        return np.nan
    
    # Calculate quantiles and scores

    scores = []
    for q, p in zip(percentiles, probs):
        if actual < q:
            loss = (1 - p) * abs(q - actual)
        else:
            loss = p * abs(q - actual)
        scores.append(loss)
    
    return 2 * np.nanmean(scores) if na_rm else 2 * np.mean(scores)

In [42]:
google_stock = pd.read_csv("./data/google_stock.csv", index_col="Date", parse_dates=True)
google_stock.tail()

,Unnamed: 0,Symbol,Open,High,Low,Close,Adj_Close,Volume,day
Date,,,,,,,,,
2018-12-24,1002,GOOG,973.900024,1003.539978,970.109985,976.219971,976.219971,1590300,1002
2018-12-26,1003,GOOG,989.010010,1040.000000,983.000000,1039.459961,1039.459961,2373300,1003
2018-12-27,1004,GOOG,1017.150024,1043.890015,997.000000,1043.880005,1043.880005,2109800,1004
2018-12-28,1005,GOOG,1049.619995,1055.560059,1033.099976,1037.079956,1037.079956,1414800,1005
2018-12-31,1006,GOOG,1050.959961,1052.699951,1023.590027,1035.609985,1035.609985,1493300,1006


In [52]:
train = google_stock[:"2016-01-04"]["Close"]
train.tail()

Date
2015-12-28    762.510010
2015-12-29    776.599976
2015-12-30    771.000000
2015-12-31    758.880005
2016-01-04    741.840027
Name: Close, dtype: float64

In [56]:
train = google_stock[:"2015-12-31"]["Close"].to_numpy()
test = google_stock[:"2016-01-04"]["Close"].to_numpy()[-1]
test

np.float64(741.840027)

In [74]:
nf1 = Naive1()
nf1.fit(train)
pred, interval = nf1.predict(horizon=1, return_predict_int=True, alpha=[0.2])
pred, interval


(array([758.880005]), [array([[744.53997703, 773.22003297]])])

In [77]:
quantile_score(interval[0][0], test, probs=[0.1]).round(2)

[744.53997703 773.22003297]
[747.40798262]


np.float64(10.02)

In [79]:
loss = (1 - 0.1) * abs(744.539 - test)
loss * 2

np.float64(4.858151400000042)

In [63]:
from forecast_tools.metrics import winkler_score

In [65]:
winkler_score(interval[0], test, alpha=0.2)

np.float64(55.679556216310175)